# Synch drive

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.07 ms (started: 2021-06-09 14:43:17 +00:00)


## Change the drive to Transformer OCR

In [19]:
%cd '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/src'

/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/src
time: 4.73 ms (started: 2021-06-09 14:43:43 +00:00)


## Install Library to capture time

In [20]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.07 s (started: 2021-06-09 14:43:46 +00:00)


## Install cuda

In [ ]:
import torch
device = torch.device("cuda")
device

device(type='cuda')

time: 2.19 s (started: 2021-06-09 13:51:00 +00:00)


# TRAIN MODEL

In [50]:
!python -W ignore main.py --source='iam' --train --epochs 200 --batch_size 10 --device 'cuda' 


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f022a2c4d40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1292, in _shutdown_workers
    self._mark_worker_as_unavailable(worker_id, shutdown=True)
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1252, in _mark_worker_as_unavailable
    assert self._workers_done_event.is_set() == shutdown
  File "/usr/lib/python3.7/multiprocessing/synchronize.py", line 328, in is_set
    with self._cond:
  File "/usr/lib/python3.7/multiprocessing/synchronize.py", line 230, in __enter__
    return self._lock.__enter__()
  File "/usr/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt: 
Traceback (most recent call last)

In [52]:
!python -W ignore main.py --source='iam' --train --epochs 200 --batch_size 10 --device 'cuda' 


Model target Path: ../output/iam/checkpoint_weights.pt
Epoch: 01 learning rate[0.0001]
Time: 3m 7s
Train Loss: 4.030
Val   Loss: 168.923
Epoch: 02 learning rate[0.0001]
Time: 3m 7s
Train Loss: 4.116
Val   Loss: 168.914
Epoch: 03 learning rate[0.0001]
Traceback (most recent call last):
  File "main.py", line 145, in <module>
    run_epochs(model, criterion, optimizer, scheduler, train_loader, val_loader, args.epochs, tokenizer, target_path,device)                
  File "/content/drive/My Drive/Transformer_ocr_new/Transformer_ocr_beam/src/engine.py", line 192, in run_epochs
    train_loss,outputs = train(model, criterion, optimizer, scheduler, train_loader, tokenizer.vocab_size, device)
  File "/content/drive/My Drive/Transformer_ocr_new/Transformer_ocr_beam/src/engine.py", line 51, in train
    output = model(imgs.float(),labels_y.long()[:,:-1])
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs

# Test Single image

In [ ]:
#!python main.py --source='iam'  --image '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/data/single_images/a01-000u-00.png'  --device 'cuda'



####################################
predicted text is: A move to stop Mr . Gaitskell from
: cannot connect to X server 


In [ ]:
#!python main.py --source='iam'  --image '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/data/single_images/b08-597z-01.png'  --device 'cuda'


####################################
predicted text is: " gy latter terroyphwith he ana Anan
: cannot connect to X server 


# TEST Without Beam 

In [ ]:
# Without Beam
!python main.py --source='iam'  --test 

Model target Path: ../output/iam/checkpoint_weights.pt
source_path : ../data/iam.hdf5
Calculate Character Error Rate 0.10170204479149941, Word Error Rate 0.29473658634559724 and Sequence Error Rate 0.9059649122807018


# TEST with Beam Search

In [ ]:
# With Beam
#!python main.py --source='iam'  --test_beam 


Model target Path: ../output/iam/checkpoint_weights.pt
source_path : ../data/iam.hdf5
Candidate 0 : quite mable to explain why he shald feelEOS
Candidate 0 : mect the Decanes , and as scon as Gony hadEOS
Candidate 0 : the harses and dunk egrough to ewe uwEOS
Candidate 0 : you not killed ! ' Because we know all things , ' theEOS
Candidate 0 : with the possibility of faulty design . " He heldEOS
Candidate 0 : all due deference , Miss Deame - come off it ! EOS
Candidate 0 : St would have been acceptable to all concemedEOS
Candidate 0 : to make you understand just what happenedEOS
Candidate 0 : covrse of action should be - First to avoid theEOS
Candidate 0 : ( Slamps Department ) while sally sulled at home . EOS
Candidate 0 : Bill is goodman , and Bveno Buchis raised on late . EOS
Candidate 0 : Pnen what of the culturol life ? Did this per - EOS
Candidate 0 : uttered together . ' Do you say that you poorEOS
Candidate 0 : Sally and of course Mus Septims , for surelyEOS
Candidate 0 : did you

In [ ]:
pwd

'/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/src'

## Check if import works

In [41]:
from data.generator import DataGenerator as DataGenerator_orig, Tokenizer as Tokenize_orig



time: 1.37 ms (started: 2021-06-10 01:12:15 +00:00)


# TEST PREDICTION WITHOUT SPELL CHECK

In [53]:
import argparse
import os
import string

import cv2
import h5py
import numpy as np
import torch
import torchvision.transforms as T
from data import preproc as pp, evaluation
from data.generator import DataGenerator as DataGenerator_orig, Tokenizer as Tokenize_orig
from data.reader import Dataset
from engine import single_image_inference, LabelSmoothing, run_epochs, get_memory
from network.model import make_model
from network.predictors import Predictor

time: 9.79 ms (started: 2021-06-10 08:52:13 +00:00)


## Utils

In [54]:
  def subsequent_masking(x):
      # x: (batch_size, seq_len - 1)
      batch_size, seq_len = x.size()
      subsequent_mask = np.triu(np.ones(shape=(seq_len, seq_len)), k=1).astype('bool')
      subsequent_mask = torch.tensor(subsequent_mask).to(x.device)
      subsequent_mask = subsequent_mask.unsqueeze(0).expand(batch_size, seq_len, seq_len)
      return subsequent_mask

  PAD_TOKEN_ID = 0

  PAD_TOKEN_INDEX = 0


  PAD_TOKEN_INDEX = 0
  vocab_length = 99
  def pad_masking(x, target_len):
      # x: (batch_size, seq_len)
      batch_size, seq_len = x.size()
      padded_positions = x == PAD_TOKEN_INDEX  # (batch_size, seq_len)
      pad_mask = padded_positions.unsqueeze(1).expand(batch_size, target_len, seq_len)
      return pad_mask

  def get_memory(model, imgs):
      x = model.conv(model.get_feature(imgs))
      bs,_,H, W = x.shape
      pos = torch.cat([
              model.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
              model.row_embed[:H].unsqueeze(1).repeat(1, W, 1),
          ], dim=-1).flatten(0, 1).unsqueeze(0)

      

      #print ("x shape : ",x.shape)
      #print ("pos shape : ",pos.shape)

      memory =  model.transformer.encoder(pos + 0.1 * x.flatten(2).permute(0, 2, 1))
      #print ("Memory shape from get_memmory:",memory.shape)
      return memory

time: 24.2 ms (started: 2021-06-10 08:52:28 +00:00)


## Constants

In [55]:
input_size = (1024, 128, 1)
max_text_length = 128
charset_base = string.printable[:95]
tokenizer_orig = Tokenize_orig(chars=charset_base, max_text_length=max_text_length)   

time: 2.88 ms (started: 2021-06-10 08:52:33 +00:00)


## Calculate Model Parameters

In [31]:
  source_path = '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/data/iam.hdf5'
  target_path = '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/output/iam/checkpoint_weights.pt'
  model = make_model(tokenizer_orig.vocab_size, hidden_dim=256, nheads=4,
  num_encoder_layers=4, num_decoder_layers=4)

  #print ("Memory shape from get_memmory:",memory.shape)
  device = torch.device("cuda")
  model.to(device)
  transform = T.Compose([
          T.ToTensor()])
          

  if os.path.exists(target_path):
      model.load_state_dict(torch.load(target_path))  
      print ("Model target Path:",target_path)          
  else:            
    print('No model checkpoint found')

Model target Path: /content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/output/iam/checkpoint_weights.pt
time: 1.82 s (started: 2021-06-09 15:25:23 +00:00)


In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

time: 1.55 ms (started: 2021-06-09 15:27:21 +00:00)


In [35]:
no_params = count_parameters(model)
print (no_params)



54689030
time: 4.63 ms (started: 2021-06-09 15:27:23 +00:00)


# Test with Validation Data

In [56]:
  source_path = '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/data/iam.hdf5'
  target_path = '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/output/iam/checkpoint_weights.pt'
  model = make_model(tokenizer_orig.vocab_size, hidden_dim=256, nheads=4,
  num_encoder_layers=4, num_decoder_layers=4)

  #print ("Memory shape from get_memmory:",memory.shape)
  device = torch.device("cuda")
  model.to(device)
  transform = T.Compose([
          T.ToTensor()])
          

  if os.path.exists(target_path):
      model.load_state_dict(torch.load(target_path))  
      print ("Model target Path:",target_path)          
  else:            
    print('No model checkpoint found')
    
  model.eval()
  predicts = []
  gt = []
  print ("source_path :",source_path)
  tes_loader = torch.utils.data.DataLoader(DataGenerator_orig(source_path,charset_base,max_text_length,'valid',transform), batch_size=1, shuffle=False, num_workers=2)
  with torch.no_grad():
      for batch in tes_loader:
          src, trg = batch
          src, trg = src.cuda(), trg.cuda()
          #print ("src shape :",src.shape)
          memory = get_memory(model,src.float())

          predictor = Predictor(
              model=model,tokenizer=tokenizer_orig
          )

          for index, candidate in enumerate(predictor.predict_one(memory, num_candidates=1)):
                candidate = candidate.replace('EOS','')
                #print(f'Candidate {index} : {candidate}')
                predicts.append(candidate)
                gt.append(tokenizer_orig.decode(trg.flatten(0,1)))

  predicts = list(map(lambda x : x.replace('SOS','').replace('EOS',''),predicts))
  gt = list(map(lambda x : x.replace('SOS','').replace('EOS',''),gt))
  
  evaluate = evaluation.ocr_metrics(predicts=predicts,
                                    ground_truth=gt,
                                    norm_accentuation=False,norm_punctuation=False)
  print("Calculate Character Error Rate {}, Word Error Rate {} and Sequence Error Rate {}".format(evaluate[0],evaluate[1],evaluate[2]))

Model target Path: /content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/output/iam/checkpoint_weights.pt
source_path : /content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/data/iam.hdf5
Calculate Character Error Rate 0.08738717887935417, Word Error Rate 0.2776945948316916 and Sequence Error Rate 0.8803763440860215
time: 5min 33s (started: 2021-06-10 08:52:38 +00:00)


In [57]:
 evaluate = evaluation.ocr_metrics(predicts=predicts,
                                    ground_truth=gt,
                                    norm_accentuation=False,norm_punctuation=False)
print("Calculate Character Error Rate {}, Word Error Rate {} and Sequence Error Rate {}".format(evaluate[0],evaluate[1],evaluate[2]))

Calculate Character Error Rate 0.08738717887935417, Word Error Rate 0.2776945948316916 and Sequence Error Rate 0.8803763440860215
time: 11.8 ms (started: 2021-06-10 08:58:35 +00:00)


## Test with Test Data

In [58]:
  
  source_path = '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/data/iam.hdf5'
  target_path = '/content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/output/iam/checkpoint_weights.pt'
  model = make_model(tokenizer_orig.vocab_size, hidden_dim=256, nheads=4,
  num_encoder_layers=4, num_decoder_layers=4)

  #print ("Memory shape from get_memmory:",memory.shape)
  device = torch.device("cuda")
  model.to(device)
  transform = T.Compose([
          T.ToTensor()])
          

  if os.path.exists(target_path):
      model.load_state_dict(torch.load(target_path))  
      print ("Model target Path:",target_path)          
  else:            
    print('No model checkpoint found')
    
  model.eval()
  predicts = []
  gt = []
  print ("source_path :",source_path)
  tes_loader = torch.utils.data.DataLoader(DataGenerator_orig(source_path,charset_base,max_text_length,'test',transform), batch_size=1, shuffle=False, num_workers=2)
  with torch.no_grad():
      for batch in tes_loader:
          src, trg = batch
          src, trg = src.cuda(), trg.cuda()
          #print ("src shape :",src.shape)
          memory = get_memory(model,src.float())

          predictor = Predictor(
              model=model,tokenizer=tokenizer_orig
          )

          for index, candidate in enumerate(predictor.predict_one(memory, num_candidates=1)):
                candidate = candidate.replace('EOS','')
                #print(f'Candidate {index} : {candidate}')
                predicts.append(candidate)
                gt.append(tokenizer_orig.decode(trg.flatten(0,1)))

  predicts = list(map(lambda x : x.replace('SOS','').replace('EOS',''),predicts))
  gt = list(map(lambda x : x.replace('SOS','').replace('EOS',''),gt))
  
  evaluate = evaluation.ocr_metrics(predicts=predicts,
                                    ground_truth=gt,
                                    norm_accentuation=False,norm_punctuation=False)
  print("Calculate Character Error Rate {}, Word Error Rate {} and Sequence Error Rate {}".format(evaluate[0],evaluate[1],evaluate[2]))

Model target Path: /content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/output/iam/checkpoint_weights.pt
source_path : /content/drive/MyDrive/Transformer_ocr_new/Transformer_ocr_beam/data/iam.hdf5
Calculate Character Error Rate 0.09779832447362631, Word Error Rate 0.2840193970448166 and Sequence Error Rate 0.8982456140350877
time: 10min 21s (started: 2021-06-10 08:59:12 +00:00)


In [48]:
 evaluate = evaluation.ocr_metrics(predicts=predicts,
                                    ground_truth=gt,
                                    norm_accentuation=False,norm_punctuation=False)
print("Calculate Character Error Rate {}, Word Error Rate {} and Sequence Error Rate {}".format(evaluate[0],evaluate[1],evaluate[2]))

Calculate Character Error Rate 0.09880445631331485, Word Error Rate 0.2862633628599899 and Sequence Error Rate 0.9045614035087719
time: 21.9 ms (started: 2021-06-10 01:26:48 +00:00)


# TEST PREDICTION WITH SPELL CHECK

## Chenge to Spell Check directory

In [59]:
%cd "/content/drive/MyDrive/spelling-correction/src"

/content/drive/MyDrive/spelling-correction/src
time: 5.68 ms (started: 2021-06-10 09:13:55 +00:00)


## Spell Check Resorce Loading

In [60]:
import os
import datetime
import string

# define parameters
source = "iam"
mode = "luong"
epochs = 1000
batch_size = 64

# define paths
data_path = os.path.join("/content/drive/MyDrive/spelling-correction", "data")
source_path = os.path.join(data_path, f"{source}.txt")

output_path = os.path.join("/content/drive/MyDrive/spelling-correction", "output", source, mode)
target_path = os.path.join(output_path, "checkpoint_weights.hdf5")
os.makedirs(output_path, exist_ok=True)

# define number max of chars per line and list of valid chars
max_text_length = 128
charset_base = string.printable[:95]
charset_special = """ÀÁÂÃÄÅÇÈÉÊËÌÍÎÏÑÒÓÔÕÖÙÚÛÜÝàáâãäåçèéêëìíîïñòóôõöùúûüý"""

print("output", output_path)
print("target", target_path)
print("charset:", charset_base + charset_special)

output /content/drive/MyDrive/spelling-correction/output/iam/luong
target /content/drive/MyDrive/spelling-correction/output/iam/luong/checkpoint_weights.hdf5
charset: 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ ÀÁÂÃÄÅÇÈÉÊËÌÍÎÏÑÒÓÔÕÖÙÚÛÜÝàáâãäåçèéêëìíîïñòóôõöùúûüý
time: 634 ms (started: 2021-06-10 09:13:57 +00:00)


In [61]:
from data2.generator2 import DataGenerator as DataGenerator2

dtgen = DataGenerator2(source=source_path,
                      batch_size=64,
                      charset=(charset_base + charset_special),
                      max_text_length=128)

print(f"Train sentences: {dtgen.size['train']}")
print(f"Validation sentences: {dtgen.size['valid']}")
print(f"Test sentences: {dtgen.size['test']}")

Train sentences: 145314
Validation sentences: 16145
Test sentences: 32
time: 6.68 s (started: 2021-06-10 09:14:03 +00:00)


## Load Sell check Model Trained Param

In [62]:
from data2 import preproc as pp, evaluation as ev
from tool.seq2seq import Seq2SeqAttention
from tool.transformer import Transformer
target_path ='/content/drive/MyDrive/spelling-correction/output/iam/luong/checkpoint_weights.hdf5'
spell_check_model = Seq2SeqAttention(dtgen.tokenizer,
                             mode,
                             units=512,
                             dropout=0.2,
                             stop_tolerance=20,
                             reduce_tolerance=15)

spell_check_model.compile(learning_rate=0.001)
#spell_check_model.summary(output_path, "summary.txt")

# get default callbacks list and load checkpoint weights file (HDF5) if exists 
spell_check_model.load_checkpoint(target=target_path)

callbacks = spell_check_model.get_callbacks(logdir=output_path, checkpoint=target_path, verbose=1)

time: 15 s (started: 2021-06-10 09:14:14 +00:00)


In [63]:
from data2 import preproc as pp


time: 1.56 ms (started: 2021-06-10 09:14:33 +00:00)


## APPLY SPELL CHECK

In [64]:
input_list =predicts

def next_test_batch(input_list):
  inputs = dtgen.prepare_sequence(sentences=input_list, sos=True, eos=True)
  yield [inputs]

predicts_spell_correct = spell_check_model.predict(x = next_test_batch(input_list))
predicts_spell_correct = [pp.text_standardize(x) for x in predicts_spell_correct]


time: 49.4 s (started: 2021-06-10 09:14:36 +00:00)


In [65]:
evaluate = evaluation.ocr_metrics(predicts=predicts_spell_correct,
                                  ground_truth=gt,)
 
print("Calculate Character Error Rate {}, Word Error Rate {} and Sequence Error Rate {}".format(evaluate[0],evaluate[1],evaluate[2]))

Calculate Character Error Rate 0.12219987699270632, Word Error Rate 0.23829569460230288 and Sequence Error Rate 0.8245614035087719
time: 19.8 ms (started: 2021-06-10 09:15:37 +00:00)
